In [ ]:
import scipp as sc
import numpy as np

In [ ]:
def to_bin_centers(d, dim):
    edges = d.coords[dim].copy()
    del d.coords[dim]
    d.coords[dim] = 0.5 * (edges[dim, 1:] + edges[dim, :-1])

In [ ]:
def to_bin_edges(d, dim):
    centers = d.coords[dim].copy()
    del d.coords[dim]
    first = 1.5*centers[dim, 0] - 0.5*centers[dim, 1]
    last = 1.5*centers[dim, -1] - 0.5*centers[dim, -2]
    bulk = 0.5 * (centers[dim, 1:] + centers[dim, :-1])
    edges = sc.concatenate(first, bulk, 'wavelength')
    edges = sc.concatenate(edges, last, 'wavelength')
    d.coords[dim] = edges

In [ ]:
path = '/home/simon/code/loki_tube_scripts/test/test_data'
direct_beam_file = 'DirectBeam_20feb_full_v3.dat'
moderator_file = 'ModeratorStdDev_TS2_SANS_LETexptl_07Aug2015.txt'
sample_run_number = 49338
sample_transmission_run_number = 49339
background_run_number = 49334
background_transmission_run_number = 49335

def load_larmor(run_number):
    return sc.neutron.load(filename=f'{path}/LARMOR000{run_number}.nxs')

def load_rkh(filename):
    return sc.neutron.load(
        filename=filename,
        mantid_alg='LoadRKH',
        mantid_args={'FirstColumnValue':'Wavelength'})
    

In [ ]:
sample_trans = load_larmor(sample_transmission_run_number)
sample = load_larmor(sample_run_number)
background_trans = load_larmor(background_transmission_run_number)
background = load_larmor(background_run_number)

In [ ]:
sample_pos_offset = sc.Variable(
    value=[0.0, 0.0, 0.30530],
    unit=sc.units.m,
    dtype=sc.dtype.vector_3_float64)
bench_pos_offset = sc.Variable(
    value=[0.0, 0.001, 0.0],
    unit=sc.units.m,
    dtype=sc.dtype.vector_3_float64)
for item in [sample, sample_trans, background, background_trans]:
    item.coords['sample_position'] += sample_pos_offset
    item.coords['position'] += bench_pos_offset

In [ ]:
def q1d(data, transmission):
    # TODO
    #transWs = self._setupAndCalculateTransmission(transWsName, "transWs")
    # TODO
    #dataWs = self._applyMask(dataWs)
    data = sc.neutron.convert(data, 'tof', 'wavelength')
    # TODO fix `num` to match Mantid's Params='0.9,-0.025,13.5'
    wavelength_bins = sc.Variable(
        dims=['wavelength'],
        unit=sc.units.angstrom,
        values=np.geomspace(0.9, 13.5, num=100))
    data = sc.rebin(data, 'wavelength', wavelength_bins)

    monitor = data.attrs['monitor1'].value.copy() # TODO is this the correct one?
    # TODO
    # monWs = CalculateFlatBackground(InputWorkspace=monWs, StartX=40000, EndX=99000, Mode='Mean')
    sc.neutron.convert(monitor, 'tof', 'wavelength', out=monitor)
    monitor = sc.rebin(monitor, 'wavelength', wavelength_bins)

    # this factor seems to be a fudge factor. Explanation pending.
    factor = 100.0 / 176.71458676442586
    data *= factor

    # Setup direct beam and normalise to monitor. I.e. adjust for efficiency of detector across the wavelengths.
    direct_beam = load_rkh(filename=f'{path}/{direct_beam_file}')
    to_bin_edges(direct_beam, 'wavelength')
    direct_beam = sc.rebin(direct_beam, 'wavelength', monitor.coords['wavelength'])
    direct_beam = monitor * transmission * direct_beam

    # Estimate qresolution function
    moderator = load_rkh(filename=f'{path}/{moderator_file}')
    to_bin_edges(moderator, 'wavelength')
    # TODO
    #qResWs = TOFSANSResolutionByPixel(InputWorkspace=dataWs,
    #                                  DeltaR=8,
    #                                  SampleApertureRadius=4.0824829046386295,
    #                                  SourceApertureRadius=14.433756729740645,
    #                                  SigmaModerator=modWs, CollimationLength=5,
    #                                  AccountForGravity=True,
    #                                  ExtraLength=2)

    q_bins = sc.Variable(
        dims=['Q'],
        unit=sc.units.one/sc.units.angstrom,
        values=np.geomspace(0.0045, 0.7, num=100)) # TODO match '0.0045,-0.08,0.7'
    # TODO QResolution
    # TODO solid angle weighting
    # TODO no pixelAdj, maybe need to broadcast direct_beam for `sum` to give correct result
    d = sc.Dataset({'data':data, 'norm':direct_beam})
    to_bin_centers(d, 'wavelength')
    d = sc.neutron.convert(d, 'wavelength', 'Q') # TODO no gravity yet
    d = sc.histogram(d, reference.coords['Q'])
    d = sc.sum(d, 'spectrum')
    I = d['data']/d['norm']

    return I


In [ ]:
sample_q1d = q1d(data=sample, transmission=sample_trans)
background_q1d = q1d(data=background, transmission=background_trans)
reduced = (sample_q1d - background_q1d)['Q', 2:-2] # TODO params

reduced.attrs['UserFile'] = sc.Variable(
    value='USER_Raspino_191E_BCSLarmor_24Feb2020_v1.txt')
reduced.attrs['Transmission'] = sc.Variable(
    value=str(self.sampleTransRun) + '_trans_sample_0.9_13.5_unfitted')
reduced.attrs['TransmissionCan'] = sc.Variable(
    value='49335_trans_can_0.9_13.5_unfitted')